In [251]:
import torch as th 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import pytorch_lightning as pl 
from pytorch_lightning import Trainer, LightningModule, LightningDataModule

import torchdyn 
from torchdyn.core import NeuralODE

import os
import cv2 
import numpy as np 
import matplotlib.pyplot as plt 

import warnings 
warnings.filterwarnings("ignore")

## DATA

In [253]:
class CircuitFused(Dataset):
    def __init__(self, datadir, image_size):
        super().__init__()
        self.datadir = datadir
        self.image_size = image_size
        self.images, self.labels = self.datareader()

    def datareader(self):
        X = []
        Y = []
        folders = os.listdir(self.datadir)
        for c, folder in enumerate(folders):
            try:
                files = os.listdir(self.datadir+folder)
                for file in files:
                    image = cv2.imread(self.datadir+folder+"/"+file)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    image = cv2.resize(image, (self.image_size[0], self.image_size[1]), cv2.INTER_AREA)/255.0
                    image = np.array(image, dtype= np.float32)
                    X.append(image.reshape(1, self.image_size[0], self.image_size[1]))
                    arr = np.zeros(15, dtype="float32")
                    arr[int(c)]=1
                    Y.append(arr)
            except:
                pass
        return X, Y

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        label = th.from_numpy(self.labels[idx])
        image = th.from_numpy(self.images[idx])
        sample = {"Image": image.reshape(1, 64, 64), "Label": label}
        return sample

## MODELS

In [254]:
class CNNODEBlock(nn.Module):
    def __init__(self, infilter:int, outfilter:int, kernel:int, moment:float):
        super(CNNODEBlock, self).__init__()
        self.conv1 = nn.Conv2d(infilter, outfilter, kernel)
        self.act = nn.ReLU()
        self.conv2 = nn.Conv2d(outfilter, infilter, kernel)
        pad = int((kernel-1)/2)
        self.pad = nn.ZeroPad2d(pad)
        self.norm1 = nn.BatchNorm2d(outfilter, momentum=moment)
        self.norm2 = nn.BatchNorm2d(infilter, momentum=moment)

    def forward(self, x):
        x = self.act(self.norm1(self.pad(self.conv1(x))))
        return self.act(self.norm2(self.pad(self.conv2(x))))

In [255]:
class UPBlock(nn.Module):
    def __init__(self, infilter:int, outfilter:int, kernel:int, moment:float):
        super(UPBlock, self).__init__()
        self.conv1 = nn.Conv2d(infilter, outfilter, kernel)
        self.norm1 = nn.BatchNorm2d(outfilter, momentum=moment)

        self.conv2 = nn.Conv2d(outfilter, outfilter, kernel)
        self.norm2 = nn.BatchNorm2d(outfilter, momentum=moment)

        self.act = nn.ReLU()
        self.pad = nn.ZeroPad2d(int((kernel-1)/2.0))
        self.pool = nn.MaxPool2d(2, 2)
    
    def forward(self, x):
        x = self.act(self.norm1(self.pad(self.conv1(x))))
        x = self.act(self.norm2(self.pad(self.conv2(x))))
        return self.pool(x)

In [256]:
class DenseBlock(nn.Module):
    def __init__(self, filter:int, dense:int, drop:float, classes:int):
        super(DenseBlock, self).__init__()
        self.pool = nn.AdaptiveAvgPool2d((1,1))
        self.flat = nn.Flatten()
        self.dense = nn.Linear(filter, dense)
        self.drop = nn.Dropout(drop)
        self.final = nn.Linear(dense, classes)
    
    def forward(self,x):
        x = self.dense(self.drop(self.flat(self.pool(x))))
        return F.softmax(self.final(self.drop(F.relu(x))), dim=-1)

In [257]:
class CNNODE(nn.Module):
    def __init__(self, num=3, filter=64, dense=256, classes=15, gf=2, kernel=3, moment=0.9, drop=0.2):
        super(CNNODE, self).__init__()
        self.cnnode = nn.ModuleList([])
        self.upsamp = nn.ModuleList([])
        self.final = DenseBlock(filter*pow(gf, num), dense, drop, classes)
        self.conv = UPBlock(1, filter, kernel, moment)
        
        for _ in range(num):
            f = CNNODEBlock(filter, filter*2, kernel, moment)
            model = NeuralODE(f, sensitivity='adjoint', solver='rk4', solver_adjoint='dopri5', atol_adjoint=1e-4, rtol_adjoint=1e-4)
            self.cnnode.append(model)
            self.upsamp.append(UPBlock(filter, filter*2, kernel, moment))
            filter = filter*gf

    def forward(self, x, t_span):
        x = self.conv(x)
        for neuralode, neuralnetwork in zip(self.cnnode, self.upsamp):
            t, x = neuralode(x, t_span)
            x = neuralnetwork(x[-1])
        return self.final(x)  

## TRAIN

In [258]:
class Learner(LightningModule):
    def __init__(self, data:Dataset, model:nn.Module, t_span:th.tensor,  batchsize:int=20, learning_rate:float=1e-4):
        super(Learner, self).__init__()
        self.model, self.t_span = model, t_span
        self.data = data
        self.batchsize = batchsize
        self.learning_rate = learning_rate

    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x = batch["Image"]
        y = batch["Label"]
        y_hat = self.model(x, self.t_span)
        loss = nn.CrossEntropyLoss()(y_hat, y)
        return {'loss': loss}
    
    def configure_optimizers(self):
        return th.optim.Adam(self.model.parameters(), lr=self.learning_rate)
    
    def train_dataloader(self):
        return DataLoader(self.data, batch_size=self.batchsize, shuffle=True)

In [259]:
PATH = r"C:\Users\suyash\Desktop\CIrcuitSolver/"
model = CNNODE()
t_span = th.linspace(0, 1, 10)
data = CircuitFused(PATH, (64, 64))
learn = Learner(data, model, t_span, 25, 1e-4)

Your vector field callable (nn.Module) should have both time `t` and state `x` as arguments, we've wrapped it for you.
Your vector field callable (nn.Module) should have both time `t` and state `x` as arguments, we've wrapped it for you.
Your vector field callable (nn.Module) should have both time `t` and state `x` as arguments, we've wrapped it for you.


In [260]:
trainer = pl.Trainer(min_epochs=200, max_epochs=300)
trainer.fit(learn)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: c:\Users\suyash\Desktop\KACHRA\Geekhore\CNNode\lightning_logs

  | Name  | Type   | Params
---------------------------------
0 | model | CNNODE | 7.9 M 
---------------------------------
7.9 M     Trainable params
0         Non-trainable params
7.9 M     Total params
31.695    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

torch.Size([25, 512, 4, 4])
torch.Size([25, 256])
